# CLSS Template Import Process

This notebook is designed to import CLSS templates and related data from specified files into the ArcGIS Online feature services. The imported data can be used to update or initialize the CLSS feature services with the latest templates and configurations.

#### Prerequisites
Before running this notebook, please ensure the following:

1. **Export Process**:  Before running the CLSS Template Import process, you must first use the 'template-export' notebook to export the CLSS templates. The exported templates (csv files) must be saved to the `/home/export` folder.

#### Steps
1. **Select Feature Service**: Use the dropdown menu to select the target CLSS feature service for importing the templates.
2. **Import Data**: The notebook will import the CLSS templates and related data into the selected feature service.
3. **Save Imported Data**: The imported data will be saved in the feature service for further use.


#### Links
- [DevOps Story #1779](https://dev.azure.com/ghinternational/GHIS/_workitems/edit/1779)


## Import Dependencies & set key variables

In [1]:
# Standard library imports
import os
import json

# Third-party library imports
import pandas as pd
from arcgis.gis import GIS
import ipywidgets as widgets
from datetime import datetime

In [2]:
# Set parameters

### if target_feature_service_id is None, the script will let the user pick from a list of feature services 
### in the org matching search term


# target_feature_service_id = '980b62dffcb94c10bff227e9a8a43fce'  # Replace with the actual target feature service ID
target_feature_service_id = 'None'
clss_search_term = 'title: CLSS'

In [3]:
# Get the current date-time
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Check if /home folder exists, if not create it
home_dir = 'home'
if not os.path.exists(home_dir):
    os.makedirs(home_dir)

# Check if ../export folder exists, if not create it
export_dir = f'{home_dir}/export'
if not os.path.exists(export_dir):
    os.makedirs(export_dir)

# gis = GIS("home")
# Following section not needed when signing in via ArcGIS Online Notebook
# Get username and password from config file 
with open(f'{home_dir}/config.json') as config_file:
    config = json.load(config_file)

AGO_username = config['AGO_username']
AGO_password = config['AGO_password']

# Sign into the ArcGIS Online or Enterprise Portal
gis = GIS("https://ghis.maps.arcgis.com/", AGO_username, AGO_password)

# These are the CLSS ArcGIS online Service Endpoints 
clssServices = [('None', 'None'),  # Example ID for Dev
             ('CLSS Dev', '4cec9a93384543e0a676e3ad892362bb'),  
             ('CLSS Test', 'f726c24e77d5442c9a9f456eec62ae5d'),  
             ('CLSS Demo', 'c5aea531e01d49358b73123e334b4c0a')]


In [4]:

if target_feature_service_id == 'None':
    # Search for feature services containing 'CLSS' in their name
    search_results = gis.content.search(query=clss_search_term, 
                                        item_type="Feature Service", 
                                        max_items=50, 
                                        sort_field="modified", 
                                        sort_order="desc")

    # Create a dictionary of search results with title as key and id as value
    clss_feature_services = {item.title: item.id for item in search_results}

    # Create a dropdown widget for selecting the feature service
    feature_service_choice = widgets.Dropdown(
        options=['None'] + list(clss_feature_services.keys()),
        description='Select Service:',
        disabled=False,
    )

    # Display the dropdown widget
    display(feature_service_choice)

    # Function to handle the feature service selection
    def on_feature_service_selected(change):
        global target_feature_service_id 
        global target_feature_service
        selected_service = change['new']
        if selected_service != 'None':
            target_feature_service_id = clss_feature_services[selected_service]
            print(f"Selected feature service ID: {target_feature_service_id}")
            target_feature_service = gis.content.get(target_feature_service_id)


    # Attach the handler to the dropdown widget
    feature_service_choice.observe(on_feature_service_selected, names='value')

else:
    print(f"Using target feature service ID: {target_feature_service_id}")
    target_feature_service = gis.content.get(target_feature_service_id)

    


Dropdown(description='Select Service:', options=('None', 'CLSS_FeatureService_Tst 2025_05', 'CLSS_Lucky_Gulch_…

In [64]:
target_hazards_table = None
target_template_table = None    
target_org_lyr = None
# Loop through the tables and layers to find the required ones
for table in target_feature_service.tables:
    if (table.properties.name == 'Hazard'):
        target_hazards_table = table
    if (table.properties.name == 'Template'):
        target_template_table = table

for layer in target_feature_service.layers:
    if (layer.properties.name == 'Organization'):
        target_org_lyr = layer
# Check if the required tables and layers were found

if target_hazards_table is None or target_template_table is None or target_org_lyr is None:
    raise ValueError("Required tables and layers not found in the feature service.")
else:
    print("Required tables and layers found in the feature service.")

Required tables and layers found in the feature service.


In [62]:
target_org_lyr

<FeatureLayer url:"https://services3.arcgis.com/j2a3SeWN04oskFYa/arcgis/rest/services/CLSS_FeatureService_Tst_2025_05/FeatureServer/2">

## Define Functions

In [ ]:
def return_indicator_json(template_indicator_df):
    # Create JSON representation of template_info_df
    template_json = []
    for lifeline, lifeline_group in template_indicator_df.groupby('Lifeline'):
        lifeline_dict = {
            "name": lifeline,
            "title": lifeline,
            "componentTemplates": []
        }
        for component, component_group in lifeline_group.groupby('Component'):
            component_dict = {
                "name": component,
                "title": component,
                "indicators": []
            }
            for _, row in component_group.iterrows():
                indicator_dict = {
                    "name": row['Indicator'],
                    "templateName": "Template Name",
                    "lifelineName": lifeline,
                    "componentName": component,
                    "weight": row['Weight']
                }
                component_dict["indicators"].append(indicator_dict)
            lifeline_dict["componentTemplates"].append(component_dict)
        template_json.append(lifeline_dict)
    return template_json

def add_to_feature_service(template_info_df, template_json):
    #### Working - add template
    global df
    df = template_info_df.rename(columns={
        'Template Name': 'Name',
        'Template Description': 'Description',
        'Status': 'Status',
        'HazardName': 'HazardID',  # Placeholder until we assign actual ID
        'OrganizationName': 'OrganizationID',  # Placeholder until we assign actual ID
    })
    try:
        df = df.drop(columns=['CreationDate', 'EditDate']).fillna('N/A') 
    except KeyError:
        print("Columns 'CreationDate' and 'EditDate' do not exist in the DataFrame.")
    
    # Add 'Content' column to the DataFrame and populate it with template_json
    df['Content'] = json.dumps(template_json, ensure_ascii=False)

    # Query Hazards Table - see if any hazards match HazardName field. If not, add new Hazard to Hazards table and get GlobalID.
    # Get raw values from the original template_info_df (unrenamed)
    hazardName = str(template_info_df['HazardName'].iloc[0]).strip()
    orgName = str(template_info_df['OrganizationName'].iloc[0]).strip()

    # Validate if they are usable (i.e., not blank or 'N/A')
    valid_hazard = hazardName and hazardName.upper() != 'N/A'
    valid_org = orgName and orgName.upper() != 'N/A'

    df['HazardID'] = getHazard(hazardName, target_hazards_table) if valid_hazard else ''
    df['OrganizationID'] = getOrg(orgName, target_org_lyr) if valid_org else ''

    # Convert the DataFrame to a list of dictionary features adhering to ArcGIS feature JSON structure
    for _, row in df.iterrows():
        if len(str(row['OrganizationID'])) > 1:
            features = {
                "attributes": {
                    "OrganizationID": row['OrganizationID'],
                    "HazardID": row['HazardID'],
                    "Name": row['Name'],
                    "Description": row['Description'],
                    "Status": row['Status'],
                    # "IsDeleted": row['IsDeleted'],
                    "Content": json.dumps(template_json, ensure_ascii=True).replace('"', '\\"')
                }
            }
        else:
            features = {
                "attributes": {
                    "HazardID": row['HazardID'],
                    "Name": row['Name'],
                    "Description": row['Description'],
                    "Status": row['Status'],
                    # "IsDeleted": row['IsDeleted'],
                    "Content": json.dumps(template_json, ensure_ascii=True).replace('"', '\\"')
                }
            }

    # Add the features to the target template table
    try:
        response = target_template_table.edit_features(adds=[features])
        print(f'Successfully uploaded template to feature service: {target_feature_service.title}')
    except Exception as e:
        print(f'Error uploading CSV to feature service: {e}')

# Query the hazards table to check if the hazardName exists
def getHazard(hazName, target_hazards_table):
    print('getHazard:',hazName)
    try:
        query_result = target_hazards_table.query(where=f"Name='{hazName}'")
        hazard_global_id = query_result.features[0].attributes['GlobalID']
    except:
        new_hazard = {
            "attributes": {
                "Name": hazName,
                "Description": "Added by script",
                "Status": 1,
                "IsDeleted": 0
            }
        }
        add_result = target_hazards_table.edit_features(adds=[new_hazard])
        if add_result['addResults'][0]['success']:
            hazard_global_id = add_result['addResults'][0]['globalId']
        else:
            raise RuntimeError("Failed to add new hazard to the hazards table")

    return hazard_global_id


# Query the Org layer to check if the hazardName exists
def getOrg(orgName, target_org_lyr):
    if pd.isna(orgName) or str(orgName).strip() == '':
        raise ValueError("OrganizationName is missing or invalid")

    orgName_safe = str(orgName).replace("'", "''")
    query_result = target_org_lyr.query(where=f"Name='{orgName_safe}'")

    if query_result.features:
        org_global_id = query_result.features[0].attributes['GlobalID']
    else:
        print(f"No org found for '{orgName}' — adding new one.")
        new_org = {
            "attributes": {
                "Name": orgName,
                "Description": "Added by script",
                "Status": 1,
                "IsDeleted": 0
            }
        }
        add_result = target_org_lyr.edit_features(adds=[new_org])
        if add_result['addResults'][0]['success']:
            org_global_id = add_result['addResults'][0]['globalId']
            print(f"Added new org '{orgName}', GlobalID: {org_global_id}")
        else:
            raise RuntimeError("Failed to add new organization to the org layer")

    return org_global_id

# Function to handle the CSV file selection
def on_csv_selected(change):
    global csv_file_path, template_info, template_df
    csv_file_path = os.path.join(export_dir, change['new'])
    template_info = csv_file_path
    template_df = pd.read_csv(template_info, sep=',', quoting=1)

    print(f"Selected template  file: {template_info}")

    indicator_json = return_indicator_json(template_df)
    print(indicator_json)
    add_to_feature_service(template_df, indicator_json)



## Process Template Export Files
process template files and import into destination feature service

User selects CSV
   ↓

on_csv_selected()
   ↓

Read CSV → template_df
   ↓

return_indicator_json(template_df) ──> indicator_json
   ↓
   
add_to_feature_service(template_df, indicator_json)
   ├── Clean/Rename columns
   ├── Lookup HazardID (getHazard)
   ├── Lookup OrgID (getOrg)
   ├── Build ArcGIS feature
   └── Submit via edit_features()


In [ ]:
output = widgets.Output()
display(output)

# List CSV files in the 'export' folder
csv_files = [f for f in os.listdir(export_dir) if f.endswith('.csv')]

# Create a dropdown widget for selecting the CSV file
csv_file_choice = widgets.Dropdown(
    options=['None'] + csv_files,
    description='Select CSV:',
    disabled=False,
)


# Display the dropdown widget & attach the handler
display(csv_file_choice)
csv_file_choice.observe(on_csv_selected, names='value')


Output()

Dropdown(description='Select CSV:', options=('None', 'CLSS_FeatureService_Dev_ACA Earthquake Template_Indicato…

In [44]:
target_hazards_table


<Table url:"https://services3.arcgis.com/j2a3SeWN04oskFYa/arcgis/rest/services/CLSS_FeatureService_Tst_2025_05/FeatureServer/4">